In [62]:
import random
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u

def generate_scattered_lines_hms_dms(
    center_ra_deg,
    center_dec_deg,
    n_lines=10,
    scatter_arcsec=5.0,
    id_prefix="src",
    number_generator=None,
):
    """
    Generate lines with RA/Dec around a center, formatted as hh:mm.mm ±dd:mm.mm.

    Parameters
    ----------
    center_ra_deg : float
        Central RA in degrees.
    center_dec_deg : float
        Central Dec in degrees.
    n_lines : int
        Number of lines to generate.
    scatter_arcsec : float
        Max positional scatter in arcseconds.
    id_prefix : str
        Source ID prefix.
    number_generator : callable or None
        Function to generate 4 numbers. If None, use random values.

    Returns
    -------
    list of str
        Lines in the form: id hh:mm.mm ±dd:mm.mm num1 num2 num3 num4
    """
    center = SkyCoord(ra=center_ra_deg * u.deg, dec=center_dec_deg * u.deg, frame="icrs")
    lines = []

    for i in range(n_lines):
        # Scatter around center position
        theta = np.random.uniform(0, 2 * np.pi)
        r = np.random.uniform(0, scatter_arcsec / 3600.0)  # in degrees
        # delta_ra = r * np.cos(theta) / np.cos(np.deg2rad(center_dec_deg))
        delta_ra = 0
        delta_dec = r * np.sin(theta)
        # delta_dec = 0

        scattered = SkyCoord(
            ra=(center.ra.deg + delta_ra) * u.deg,
            dec=(center.dec.deg + delta_dec) * u.deg,
            frame="icrs"
        )

        # RA: convert degrees → hours, then format as hh:mm.mm
        ra_hours = scattered.ra.hour
        ra_h = int(ra_hours)
        ra_m = (ra_hours - ra_h) * 60
        ra_str = f"{ra_h:02d}:{ra_m:05.2f}"

        # Dec: format as ±dd:mm.mm
        dec_deg_total = scattered.dec.deg
        sign = "+" if dec_deg_total >= 0 else "-"
        dec_deg = int(abs(dec_deg_total))
        dec_min = (abs(dec_deg_total) - dec_deg) * 60
        dec_str = f"{sign}{dec_deg:02d}:{dec_min:05.2f}"

        # Generate numbers
        if number_generator is None:
            nums = [f"{random.uniform(0, 1):.3f}" for _ in range(4)]
        else:
            nums = [f"{x:.3f}" for x in number_generator(i)]

        dist = 80000*(0.9+0.2*np.random.random_sample())

        line = f"{id_prefix}{i} {ra_str} {dec_str} {dist} {' '.join(nums)}"
        lines.append(line)

    return lines

In [ ]:
lines = generate_scattered_lines_hms_dms(
    center_ra_deg=267.0,
    center_dec_deg=-18.0,
    n_lines=int(2E5),
    scatter_arcsec=2.0 * 3600,
    id_prefix="123"
)

In [ ]:
with open("../bayestarInjs/host_galaxies.txt", "w") as f:
    for line in lines:
        f.writelines(line+"\n")

In [ ]:
line